### ELT-процесс: формирование витрины ежедневных трат

В этом ноутбуке реализован простой ELT-процесс, состоящий из трёх этапов:

1. Extract (извлечение данных)  
   Из таблицы transactions выбираются только расходные операции (transaction_type = 'Negative') с привязкой к категориям.  
   Дополнительно создаётся поле ymd — дата в формате год-месяц-день.

2. Transform (агрегация в нужный формат) 

   На этом этапе считается итог расхода за день. 
   В CTE transform данные группируются по:
   - названию категории (name)
   - дате (ymd)  


3. Load (загрузка результата в таблицу)  
   Готовые агрегированные данные вставляются в витринную таблицу daily_spend, которая содержит:
   - название категории
   - дату
   - сумму трат за этот день

Таким образом, код формирует удобную ежедневную витрину расходов, которую можно использовать для последующего анализа, визуализации или отчётности.


In [72]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

from sqlalchemy import create_engine

DB_PATH = "bank.db"

In [73]:
engine = create_engine("sqlite:///bank.db")
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript(
"""
CREATE TABLE IF NOT EXISTS daily_spend (
    name TEXT,
    ymd TEXT,
    total_spent REAL
);

WITH extract AS (
    SELECT 
        client_id,
        amount,
        strftime('%Y-%m-%d', date) AS ymd,
        name
    FROM 
        transactions t
    JOIN categories c ON t.product_category = c.id
    WHERE
        transaction_type = 'Negative'
), 

transform AS (
    SELECT 
        name,
        ymd,
        SUM(amount) AS total_spent
    FROM 
        extract
    GROUP BY 
        name, ymd
)

-- Load
INSERT INTO daily_spend (name, ymd, total_spent)
    SELECT 
        name,
        ymd,
        total_spent
    FROM 
        transform;
"""
)

conn.commit()
conn.close()

result = pd.read_sql("""
SELECT *
FROM 
    daily_spend  
""", engine)

result


,name,ymd,total_spent
0,Авиабилеты,2020-01-01,27358.0
1,Авиабилеты,2020-01-04,57172.0
2,Авиабилеты,2020-01-05,61852.0
3,Авиабилеты,2020-01-07,25285.0
4,Авиабилеты,2020-01-10,40300.0
...,...,...,...
106735,Цветы,2020-12-27,60839.0
106736,Цветы,2020-12-28,56732.0
106737,Цветы,2020-12-29,56824.0
106738,Цветы,2020-12-30,66607.0


In [74]:
result['name'].nunique()

29

In [75]:
# грузим витрину
df = pd.read_sql("SELECT name, ymd, total_spent FROM daily_spend", engine)
df["ymd"] = pd.to_datetime(df["ymd"])

# фильтр на 2020
df = df[(df["ymd"] >= "2020-01-01") & (df["ymd"] <= "2020-12-31")].copy()

df

,name,ymd,total_spent
0,Авиабилеты,2020-01-01,27358.0
1,Авиабилеты,2020-01-04,57172.0
2,Авиабилеты,2020-01-05,61852.0
3,Авиабилеты,2020-01-07,25285.0
4,Авиабилеты,2020-01-10,40300.0
...,...,...,...
106735,Цветы,2020-12-27,60839.0
106736,Цветы,2020-12-28,56732.0
106737,Цветы,2020-12-29,56824.0
106738,Цветы,2020-12-30,66607.0


### Динамика совокупных ежедневных трат (2020)

На графике показана динамика общей суммы ежедневных расходов по всем категориям за 2020 год.  
Визуализация позволяет оценить:
- общий уровень и вариативность дневных трат
- наличие всплесков и аномалий
- сезонные изменения в течение года

In [76]:
#  создаём пустую фигуру
fig = go.Figure()

# добавляем трек (линию) с дневными тратами
fig.add_trace(go.Scatter(
    x=daily_total["ymd"],
    y=daily_total["total_spent"],
    mode="lines",
    name="Total spend"
))

# настраиваем заголовок, оси и ползунок диапазона дат
fig.update_layout(
    title="Total Daily Spend (2020)",
    xaxis_title="Date",
    yaxis_title="Total spent",
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="date"
    )
)

# отображаем график
fig.show()

to do выводы к графикам 

### Топ категорий по суммарным расходам (2020)

Горизонтальная столбчатая диаграмма показывает топ-10 категорий по общей сумме расходов за 2020 год.  
Визуализация позволяет:
- быстро определить ключевые категории с наибольшим вкладом в общие траты
- сравнить масштабы расходов между категориями
- выделить основные направления потребительских расходов

In [77]:
TOP_N = 10

# считаем суммарные траты по категориям за весь период
top_cat = (
    df.groupby("name", as_index=False)["total_spent"]
    .sum()
    .sort_values("total_spent", ascending=False)
    .head(TOP_N)
)

# строим bar
fig = px.bar(
    top_cat,
    x="total_spent",
    y="name",
    orientation="h",
    title=f"Top {TOP_N} categories by total spend (2020)"
)

# немного улучшаем читаемость
fig.update_layout(yaxis_title="Category", xaxis_title="Total spent")
fig.show()

### Динамика ежедневных расходов по топ-категориям (2020)

Линейный график отображает ежедневную динамику расходов для топ-3 категорий с наибольшей суммарной величиной трат за 2020 год.  
Каждая линия соответствует отдельной категории.

Визуализация позволяет:
- сравнить поведение расходов между ключевыми категориями во времени
- выявить волатильность и сезонные колебания
- определить категории с наиболее стабильными и наиболее изменчивыми расходами

In [78]:
TOP_N = 3

# 1) берём топ категорий
top_names = (
    df.groupby("name")["total_spent"].sum()
    .sort_values(ascending=False)
    .head(TOP_N)
    .index
)

daily_top = df[df["name"].isin(top_names)].sort_values("ymd")

# 2) line-график (каждая категория — отдельная линия)
fig = px.line(
    daily_top,
    x="ymd",
    y="total_spent",
    color="name",
    title=f"Daily spend dynamics: Top {TOP_N} categories (2020)"
)

# 3) rangeslider для зума
fig.update_layout(xaxis=dict(rangeslider=dict(visible=True), type="date"),
                  xaxis_title="Date", yaxis_title="Total spent")
fig.show()

to do переделать график в виде bar, чтобы видеть топ категорий в каждом месяце 


### Тепловая карта ежедневных расходов: месяц × день месяца (2020)

Тепловая карта отражает суммарные ежедневные расходы по всем категориям в разрезе месяцев и дней месяца за 2020 год.  
Цвет ячейки соответствует величине совокупных трат за конкретный день.

Визуализация позволяет:
- выявить внутримесячные и межмесячные паттерны расходов
- обнаружить дни с аномально высокими или низкими тратами
- сравнить общую интенсивность расходов между месяцами

In [79]:
# heatmap: "месяц × день месяца" (сумма трат по всем категориям)
# агрегируем общий spend по дням (по всем категориям)
daily_total = (
    df.groupby("ymd", as_index=False)["total_spent"]
      .sum()
)

# признаки для тепловой карты
daily_total["month"] = daily_total["ymd"].dt.strftime("%Y-%m")
daily_total["day"] = daily_total["ymd"].dt.day

# фиксируем полный набор месяцев 2020 и дней 1..31
month_order = pd.date_range("2020-01-01", "2020-12-01", freq="MS").strftime("%Y-%m").tolist()
day_order = list(range(1, 32))

# строим матрицу и "дополняем" отсутствующие месяцы/дни нулями
heat = (
    daily_total.pivot_table(
        index="month",
        columns="day",
        values="total_spent",
        aggfunc="sum",
        fill_value=0
    )
    .reindex(index=month_order, columns=day_order, fill_value=0)
)

# heatmap
fig = px.imshow(
    heat,
    aspect="auto",
    title="Heatmap: total spend by month and day (2020)",
    labels=dict(x="Day of month", y="Month", color="Total spent")
)

# заставляем Plotly подписать все тики (месяцы и дни)
fig.update_xaxes(tickmode="array", tickvals=day_order, ticktext=[str(d) for d in day_order])
fig.update_yaxes(tickmode="array", tickvals=month_order, ticktext=month_order)

fig.show()

### Вывод

В этом скрипте реализован простой ELT-пайплайн для построения витрины ежедневных расходов:

- Extract: из transactions берём только расходные операции (transaction_type = 'Negative') и присоединяем справочник categories, чтобы получить название категории (name) и дату в формате ymd
- Load: сохраняем результат в отдельную таблицу-витрину daily_spend (name, ymd, total_spent) — так дальше не нужно каждый раз пересчитывать агрегации на “сырых” транзакциях
- Transform (для анализа): уже после загрузки витрины агрегируем её в нужных разрезах (по дням/месяцам/категориям) и строим графики

Далее на основе витрины выполнена визуальная проверка данных за 2020 год:  
- линейный график показывает общую динамику ежедневных трат
- bar-chart выделяет категории с максимальным вкладом
- график по топ-категориям позволяет сравнить их поведение во времени
- heatmap «месяц × день месяца» помогает увидеть сезонность и повторяющиеся пики расходов